In [1]:
from tokenizers import (
    decoders,
    models,
    normalizers,
    pre_tokenizers,
    processors,
    trainers,
    Tokenizer
)

Initialize a tokenizer instance

In [2]:
tokenizer = Tokenizer(models.WordPiece(unk_token='[UNK]'))

We need to set the normalization components, which are *convert to lowercase* -> *normalize with NFKC*

In [3]:
tokenizer.normalizer = normalizers.Sequence(
    [normalizers.Lowercase(), normalizers.NFKD()]
)

Next we have pretokenization, eg how do we split into words before tokenization? For Dhivehi we want to split on both whitespace and punctuation (a comma isn't part of a word - it's separate). This is a common approach and covered with:

In [4]:
tokenizer.pre_tokenizer = pre_tokenizers.Whitespace()

Next we train the tokenizer, we've already assigned the WordPiece tokenizer so all we do now is pass Dhivehi text data to the training function, specify any special tokens to include in our vocab (as they will not be found in the training data... Hopefully!), and specify our target vocab size.

In [5]:
trainer = trainers.WordPieceTrainer(
    vocab_size=30_000,
    special_tokens=['[UNK]', '[PAD]', '[CLS]', '[SEP]', '[MASK]'],
    min_frequency=2,
    continuing_subword_prefix='##'
)

---

#### Downloading the Dhivehi Corpus

The Dhivehi corpus is hosted at `https://huggingface.co/datasets/ashraq/dhivehi-corpus`, we can download it via `datasets.load_dataset`.

In [1]:
import datasets

dv = datasets.load_dataset(
    'ashraq/dhivehi-corpus',
    'train',
    streaming=True
)

dv

Using custom data configuration train


{'train': <datasets.iterable_dataset.IterableDataset at 0x7fdf21064f70>}

In [2]:
dv['train']

In [3]:
count = 0
for row in dv['train']:
    print(row)
    count += 1
    if count == 3: break

{'text': 'ފޯނަކާއި ވައިފައި އޮތިއްޔާ ބޭނުން އެއްޗެއް ކީދާނެ ތާއެވެ. އެކަމަކު، ބުރަ މަސައްކަތް ކުރާ ފަރާތްތަކުގެ މަސައްކަތުގެ ޖޯޝު ގެއްލުވާލަން ތިކުރާ ފިޑި އަމަލަކީ ވަރަށްވެސް ނާތަހުޒީބު ކަމެކެވެ. ކަމެއް ނުބައިކޮށް ދާނަމަ އެކަމާ އަޑު އުފުލަން ޖެހެއެވެ. ނަމަވެސް މީސްމީޑިޔާއަކީ މިކަން ކުރާ ހަމައެކަނި ވަސީލަތަކަށްވުމުން، މިކަން "ކަޑަ" އެވެ. އަދި އަމަލީ ގޮތުން ކަންތައް ކުރާ މީހުންގެ މަސައްކަތުގެ ކުރި މަރާލަން ތިކުރާ މަސައްކަތް އަދި މާ "ކަޑަ"އެވެ.'}
{'text': 'ރައީސް އޮފީހުގެ ޕްރެސް ސެކެޓްރީ މަސްއޫދު އިމާދު ވިދާޅުވީ، ނައިބު ރައީސް ވަހީދުއްދީން އިސްތިއުފާދެއްވި ކަމަށް މިހާރު ފެތުރިގެން އުޅޭ ހަބަރަކީ އެއްވެސް ހަގީގަތެއް ނެތް ވާހަކައެއް ކަމަށެވެ.'}
{'text': 'އިންދިރާ ގާންދީ މެމޯރިޔަލް ހޮސްޕިޓަލް - ވަގުތު ފޮޓޯ: މާހިލް މުހައްމަދު އަމިއްލަ ކްލިނިކް ތަކުން ދޫކުރާ ބޭސް ސިޓީތައް އައިޖީއެމްއެޗުގެ ބޭސްސިޓީ ތަކަށް ބަދަލު ކުރުން އޮގަސްޓް މަހުން ފެށިގެން ހުއްޓާލަން އެ ހޮސްޕިޓަލުން ނިންމައިފިއެވެ.'}


Create a generator that will yield the correct output (eg extract the value in `'text'`)

In [4]:
def dv_text():
    for row in dv['train']:
        yield row['text']

In [5]:
count = 0
for row in dv_text():
    print(row)
    count += 1
    if count == 3: break

ފޯނަކާއި ވައިފައި އޮތިއްޔާ ބޭނުން އެއްޗެއް ކީދާނެ ތާއެވެ. އެކަމަކު، ބުރަ މަސައްކަތް ކުރާ ފަރާތްތަކުގެ މަސައްކަތުގެ ޖޯޝު ގެއްލުވާލަން ތިކުރާ ފިޑި އަމަލަކީ ވަރަށްވެސް ނާތަހުޒީބު ކަމެކެވެ. ކަމެއް ނުބައިކޮށް ދާނަމަ އެކަމާ އަޑު އުފުލަން ޖެހެއެވެ. ނަމަވެސް މީސްމީޑިޔާއަކީ މިކަން ކުރާ ހަމައެކަނި ވަސީލަތަކަށްވުމުން، މިކަން "ކަޑަ" އެވެ. އަދި އަމަލީ ގޮތުން ކަންތައް ކުރާ މީހުންގެ މަސައްކަތުގެ ކުރި މަރާލަން ތިކުރާ މަސައްކަތް އަދި މާ "ކަޑަ"އެވެ.
ރައީސް އޮފީހުގެ ޕްރެސް ސެކެޓްރީ މަސްއޫދު އިމާދު ވިދާޅުވީ، ނައިބު ރައީސް ވަހީދުއްދީން އިސްތިއުފާދެއްވި ކަމަށް މިހާރު ފެތުރިގެން އުޅޭ ހަބަރަކީ އެއްވެސް ހަގީގަތެއް ނެތް ވާހަކައެއް ކަމަށެވެ.
އިންދިރާ ގާންދީ މެމޯރިޔަލް ހޮސްޕިޓަލް - ވަގުތު ފޮޓޯ: މާހިލް މުހައްމަދު އަމިއްލަ ކްލިނިކް ތަކުން ދޫކުރާ ބޭސް ސިޓީތައް އައިޖީއެމްއެޗުގެ ބޭސްސިޓީ ތަކަށް ބަދަލު ކުރުން އޮގަސްޓް މަހުން ފެށިގެން ހުއްޓާލަން އެ ހޮސްޕިޓަލުން ނިންމައިފިއެވެ.


---

In [11]:
tokenizer.train_from_iterator(dv_text(), trainer=trainer)

In [12]:
tokenizer.get_vocab_size()

30000

After training the tokenizer we need to set the post-processing steps, eg add any special tokens. At the start and end of every sequence we add a classifier `[CLS]` token and a seperator `[SEP]` token. We will be adding these using their token IDs which we can find with:

In [13]:
cls_id = tokenizer.token_to_id('[CLS]')
sep_id = tokenizer.token_to_id('[SEP]')

To set the post processing template we use something called the `TemplateProcessing`. This allows us to specify how to deal with both single sentences and sentence pairs (which we will need during NSP pretraining). The first/single sentence is represented by `$A` and the second (for pairs) is represented by `$B`.

Our special tokens, `$A`, and `$B` are all followed by an integer value which indicated their token type ID value.

In [14]:
tokenizer.post_processor = processors.TemplateProcessing(
    single=f'[CLS]:0 $A:0 [SEP]:0',
    pair=f'[CLS]:0 $A:0 [SEP]:0 $B:1 [SEP]:1',
    special_tokens=[
        ('[CLS]', cls_id),
        ('[SEP]', sep_id)
    ]
)

The final step is to set the `decoder` component of the tokenizer, here we just use the WordPiece tokenizer with the `##` word part prefix specified.

In [15]:
tokenizer.decoder = decoders.WordPiece(prefix='##')

Our tokenizer is now fully prepared, all that's left is to save it. To load the tokenizer with HF transformers we will first need to load our current tokenizer into a transformers fast tokenizer like so:

In [16]:
from transformers import PreTrainedTokenizerFast

full_tokenizer = PreTrainedTokenizerFast(
    tokenizer_object=tokenizer,
    unk_token='[UNK]',
    pad_token='[PAD]',
    cls_token='[CLS]',
    sep_token='[SEP]',
    mask_token='[MASK]'
)

*(Fast tokenizers are faster than the usual tokenizers because they are implemented in Rust)*

In [17]:
full_tokenizer.save_pretrained('bert-base-dv')

('bert-base-dv/tokenizer_config.json',
 'bert-base-dv/special_tokens_map.json',
 'bert-base-dv/tokenizer.json')

In [18]:
tokenizer = PreTrainedTokenizerFast.from_pretrained('jamescalam/bert-base-dv')

In [19]:
tokenizer("(ސެންޓޭ)، ނާއިޒް ހަސަން (ދާދު) އަދި ހުސައިން ނިހާންގެ އިތުރުން ސަމްދޫހު މުހައްމަދު ހިމަނާފައިވެއެވެ")

{'input_ids': [2, 9, 2290, 134, 1605, 4076, 842, 1480, 9, 1186, 314, 10, 321, 2209, 1771, 212, 781, 2287, 123, 823, 4381, 832, 674, 934, 3412, 3], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

And we're done, our tokenizer is ready.